In [ ]:
import numpy as np
import pandas as pd

training_data = pd.read_excel("training.xlsx")
prediction_data = pd.read_excel("predictions_training.xlsx")

#define a helper function to calculate IOU for one pair of boxes

def calculate_iou(boxA, boxB):
 
    #determine coordinates of the intersection rectangle
    yA = max(boxA[0], boxB[0])
    xA = max(boxA[1], boxB[1])
    yB = min(boxA[2], boxB[2])
    xB = min(boxA[3], boxB[3])

    #compute the area of intersection rectangle
    inter_width = max(0, xB - xA)
    inter_height = max(0, yB - yA)
    intersection_area = inter_width * inter_height

    #compute the area of both boxes
    boxA_area = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxB_area = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    #compute the union area
    union_area = boxA_area + boxB_area - intersection_area

    #avoid division by zero
    if union_area == 0:
        return 0

    #compute IOU
    iou = intersection_area / union_area
    return iou

#define a safer function to calculate mean IOU

def mean_iou_by_filename(true_df, pred_df):

    #keep only required columns
    t = true_df[['filename', 'min_r', 'min_c', 'max_r', 'max_c']].copy()
    p = pred_df[['filename', 'min_r', 'min_c', 'max_r', 'max_c']].copy()

    #rename prediction columns to avoid conflicts
    p = p.rename(columns={
        'min_r': 'p_min_r',
        'min_c': 'p_min_c',
        'max_r': 'p_max_r',
        'max_c': 'p_max_c'
    })

    #merge both tables on filename
    df = t.merge(p, on='filename', how='inner')

    ious = []
    for _, row in df.iterrows():
        true_box = [row['min_r'], row['min_c'], row['max_r'], row['max_c']]
        pred_box = [row['p_min_r'], row['p_min_c'], row['p_max_r'], row['p_max_c']]
        ious.append(calculate_iou(true_box, pred_box))

    mean_val = float(np.mean(ious)) if ious else 0.0
    return mean_val, ious, df[['filename']]

#run the evaluation

mean_iou_value, all_ious, filenames = mean_iou_by_filename(training_data, prediction_data)

#display results

print("Individual IOUs for each image:")
for i, (fname, val) in enumerate(zip(filenames['filename'], all_ious)):
    print(f"{i+1}. {fname} → IOU = {val:.3f}")

print("Mean IOU across all images:", round(mean_iou_value, 3))